In [1]:
import lib.draw as draw
import lib.marking_evaluation as meval
import matplotlib.pyplot as plt
import numpy as np
import copy
from random import randint, random

In [2]:
population_size = 100
limit_of_generations = 1000
fitness_limit = 10
no_stable_generations = 10

elitist_rate = 0.1
survival_rate = 0.7
mutation_rate = 0.2

In [3]:
def create_adversary(opposing_team):
    adversary = meval.Team()
    adversary.initialize_heuristic(w1=1, w2=1, w3=1, w4=1, mad=5)
    adversary.add_player(opposing_team[0][0], opposing_team[0][1], True)
    for player in opposing_team[1:]:
        adversary.add_player(player[0], player[1])
    return adversary

In [4]:
def create_team(team_coordinates):
    team = meval.Team()
    for player in team_coordinates:
        team.add_player(player[0], player[1]) 
    return team

In [5]:
def generate_random_start():
    # 16 to avoid starting inside the box
    return [[randint(16, 100), randint(0, 100)] for _ in range(10)]

In [6]:
def evaluate(population):
    population_fitnesses = []
    for individual in population:
        population_fitnesses.append(adversary.calculate_heuristic(create_team(individual)))
    return population_fitnesses

In [7]:
def selection(population, population_fitnesses):
    survival_guaranteed_above = np.quantile(population_fitnesses, 1 - elitist_rate)

    unfit_individuals = []
    i = 0
    for individual, fitness in zip(population, population_fitnesses):
        if fitness < survival_guaranteed_above:
            if survival_rate < random():
                unfit_individuals.append(i)
        i+=1
    for individual_id in reversed(unfit_individuals):
        del population[individual_id]
        del population_fitnesses[individual_id]
    
    return population, population_fitnesses

In [8]:
def crossover(population, population_fitnesses):
    while len(population) < population_size:
        parent1 = randint(0, len(population) - 1)
        parent2 = randint(0, len(population) - 1)

        individual = copy.deepcopy(population[parent1])

        crossover_players = [randint(0, 9) for i in range(randint(1, 8))]

        for player in crossover_players:
            individual[player] = population[parent2][player]

        population.append(individual)

    return population

In [9]:
def mutation(population):
    for individual in population:
        mutated_players = [randint(0, 9) for i in range(randint(0, 9))]
        for player in mutated_players:
            if random() < mutation_rate:
                individual[player] = [randint(0, 100), randint(0, 100)]

    return population

In [10]:
opposing_team = [(5, 39), (9, 28), (10, 72), (23, 98), (26, 1), (20, 49), (32, 27), (32, 70), (48, 97), (48, 51), (48, 3)]
adversary = create_adversary(opposing_team)

In [11]:
population = [generate_random_start() for _ in range(population_size)]
best_proposal = population[0]
best_heuristic = -100

for generation in range(limit_of_generations):
    population_fitnesses = evaluate(population)
    population, population_fitnesses = selection(population, population_fitnesses)
    
    best_generation_fitness = max(population_fitnesses)
    if best_generation_fitness > best_heuristic:
        best_heuristic = best_generation_fitness
        best_proposal = population[population_fitnesses.index(best_generation_fitness)]

    population = crossover(population, population_fitnesses)
    population = mutation(population)

    print(f'{generation} - Pop fitness {max(population_fitnesses)}, Max fitness {best_heuristic}')

0 - Pop fitness 2.4188253989707933, Max fitness 2.4188253989707933
1 - Pop fitness 2.3169905338024304, Max fitness 2.4188253989707933
2 - Pop fitness 2.2862025091720475, Max fitness 2.4188253989707933
